In [ ]:
#!pip install docplex

In [ ]:
from docplex.cp.model import *

mdl = CpoModel()

### Read in instance

In [ ]:
import csv
instance=[]
with open('instances/instance_base.csv', newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        
        row = ' '.join(row).split()
        #print(row)
        instance.append(row)

#instance[1]

In [ ]:
[num_containers, num_carriers, num_chassis, fixed_stack_cost, unit_cost_stack]=[int(float(i)) for i in instance[1]]
containers=[int(float(i)) for i in instance[2]]
carriers=instance[3]
release_date=[int(float(i)) for i in instance[4]]
transload_processing_time=[float(i) for i in instance[5]]
priority=[int(float(i)) for i in instance[6]]

In [ ]:
carrier_names = []
c=0
for i in range(num_carriers):
    carrier_names.append(instance[7][c])
    c+=2

In [ ]:
#initialising dictionaries for the various cost based on carriers
demurrage_free_dict=dict.fromkeys(carrier_names)
demurrage_daily_dict=dict.fromkeys(carrier_names)
detention_free_dict=dict.fromkeys(carrier_names)
detention_daily_dict=dict.fromkeys(carrier_names)

counter=1
for i in carrier_names:
    demurrage_free_dict[i]=instance[7][counter]
    demurrage_daily_dict[i]=instance[8][counter]
    detention_free_dict[i]=instance[9][counter]
    detention_daily_dict[i]=instance[10][counter]
    counter+=2

In [ ]:
demurrage_free_period=[]
demurrage_daily_cost=[]
detention_free_period=[]
detention_daily_cost=[]
for i in carriers:
    demurrage_free_period.append(int(demurrage_free_dict[i]))
    demurrage_daily_cost.append(int(demurrage_daily_dict[i]))
    detention_free_period.append(int(demurrage_free_dict[i]))
    detention_daily_cost.append(int(demurrage_daily_dict[i]))

In [ ]:
fixed_cost_stack, unit_cost_stack = int(float(instance[1][3])), int(float(instance[1][4]))
#fixed_cost_stack
leg_travel_times = dict()
for i in range(0,6,2): 
    leg_travel_times[instance[11][i]]=float(instance[11][i+1])

In [ ]:
#priority=[100,100,10,50,50,50,10,10,10,10]

### Variables:

In [ ]:
transload={}
stack={}
leg_2={}

for i in range(num_containers):
    
    transload[i] = mdl.interval_var()
    stack[i] = mdl.interval_var(optional=True)
    leg_2[i] = mdl.interval_var() #terminal to stack optional=True
    #leg_3[i] = mdl.interval_var(optional=True) #stack to transload

### Adding precendence constraints

In [ ]:
#precendences = [("leg_1","transload"), ("leg_2","stack"), ("stack, leg_3"), ('leg_3',"transload")]

In [ ]:
for i in range(num_containers):
    mdl.add(mdl.end_before_start(leg_2[i], stack[i]))
    mdl.add(mdl.end_before_start(stack[i], transload[i]))
    mdl.add(mdl.end_before_start(leg_2[i], transload[i]))
    
    mdl.add(mdl.start_of(transload[i])==mdl.end_of(stack[i]))
    mdl.add(mdl.end_of(leg_2[i])==mdl.start_of(stack[i])) 

### Adding resource constraints

In [ ]:

chassis_usage = mdl.step_at(0,0)

for i in range(num_containers):
    #the pulse function seizes 1 resource for the interval amount of time, then goes back down 
    chassis_usage += mdl.pulse(transload[i],1) 
    chassis_usage += mdl.pulse(leg_2[i],1)
    

#constraint limiting number of chassis used at any given time 
mdl.add(chassis_usage <= num_chassis)

In [ ]:
# no overlap for each container
for i in range(num_containers):
    mdl.add(mdl.no_overlap([transload[i],leg_2[i]]))

# Adding constraints end() = start() for transitioning of the same resource
for i in range(num_containers):
    mdl.add(mdl.end_of(leg_1[i])<=mdl.start_of(transload[i]))
    mdl.add(mdl.end_of(leg_2[i])<=mdl.start_of(stack[i]))
    mdl.add(mdl.end_of(stack[i])<=mdl.start_of(leg_3[i]))
    mdl.add(mdl.end_of(leg_3[i])<=mdl.start_of(transload[i]))

In [ ]:
#defining leg lengths
for i in range(num_containers):
    mdl.add(mdl.length_of(transload[i])==transload_processing_time[i]+2*1)
    mdl.add(mdl.length_of(leg_2[i])==1)
    #mdl.add(mdl.size_of(stack[i])<=10)#...assume must stay at stack if goes there
    mdl.add(mdl.start_of(transload[i])>=release_date[i])
    mdl.add(mdl.start_of(leg_2[i])>=release_date[i])
    
    #mdl.add(mdl.presence_of(stack[i])==mdl.presence_of(leg_2[i]))
    
    #mdl.add(mdl.alternative(leg_1[i],[leg_2[i]]))
    #mdl.add(mdl.sum(mdl.presence_of(leg_1[i])+mdl.presence_of(leg_2[i]))==1)
    #mdl.add(mdl.no_overlap([leg_1[i],leg_2[i],stack[i],leg_3[i],transload[i]]))

In [ ]:
#unit_cost_stack=-100

### The objective:

In [ ]:
# create the obj and add it.
mdl.add( 
    mdl.minimize( 
        mdl.sum(priority[i] *500* (mdl.end_of(transload[i]) - release_date[i]) 
                for i in range(num_containers) ) # the priority associated cost
        + mdl.sum(detention_daily_cost[i] * mdl.max([0, mdl.end_of(transload[i]) - release_date[i] - detention_free_period[i]]) 
                  for i in range(num_containers) )#detention cost, added end since container needs to be returned empty not just started
        + mdl.sum(mdl.presence_of(leg_2[i]) *(fixed_stack_cost + mdl.length_of(stack[i]) * unit_cost_stack) 
                  for i in range(num_containers) )# cost at stack, multiplying all costs by presence
        + mdl.sum(demurrage_daily_cost[i] * mdl.max([0, mdl.start_of(transload[i]) - release_date[i] - demurrage_free_period[i], 
                                                     mdl.presence_of(leg_2[i])* (mdl.start_of(leg_2[i]) - release_date[i] - demurrage_free_period[i])]) 
                  for i in range(num_containers) )# demurrage cost at terminal until leaves either on leg 1 or  
        
       # + mdl.sum(10*mdl.presence_of(leg_2[i]) for i in range(num_containers)) #gas money
      ) 
    ) 

#### Solve CP and solution

In [ ]:
# Solve the model
print("\nSolving model....")
msol = mdl.solve(TimeLimit=120)
print("done")
msol.print_solution()

In [ ]:
#help(msol)

In [ ]:
#mdl.pulse?

In [ ]:
trans,l2,s = [],[],[]
for i in range(num_containers):
    trans.append(msol.get_value(transload[i]))
    l2.append(msol.get_value(leg_2[i]))
    s.append(msol.get_value(stack[i]))

In [ ]:
trans

In [ ]:
priority

In [ ]:
l2

In [ ]:
s

In [ ]:
#mdl.alternative?

In [ ]:
import docplex.cp.utils_visu as visu
import matplotlib.pyplot as plt
%matplotlib inline
#Change the plot size
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

In [ ]:
#chassisF = CpoStepFunction()

#for i in range(num_containers):
 #   itv = msol.get_var_solution(transload[i])
  #  itv2 = msol.get_var_solution(leg_2[i])
   # chassisF.add_value(itv.get_start(), itv.get_end(), 1)
    #chassisF.add_value(itv.get_start(), itv.get_end(), 1)

visu.timeline('Solution SchedCumul')
visu.panel(name="Schedule")
for i in range(num_chassis):
    wt = msol.get_var_solution(transload[i]) 
    visu.interval(wt, 'lightblue', 'transload '+str(i))   
    wt = msol.get_var_solution(stack[i])   
    visu.interval(wt, 'blue', 's')
    wt = msol.get_var_solution(leg_2[i])   
    visu.interval(wt, 'g', 'leg_2 '+str(i))
#visu.panel(name="chassis")
#visu.function(segments=chassisF, style='area')
visu.show()

In [ ]:
for i in range(num_containers):
    visu.sequence(name=i)
    wt = msol.get_var_solution(transload[i]) 
    visu.interval(wt, 'lightblue', 'transload c'+str(i))   
    wt = msol.get_var_solution(stack[i]) 
    if wt.is_present():
        visu.interval(wt, 'lightgreen', 's')
    wt = msol.get_var_solution(leg_2[i])   
    visu.interval(wt, 'gold', 'leg_trip c'+str(i))
visu.panel(name="Schedule")
visu.show()